# Hashmaps

In [5]:
from grpc import insecure_channel
from torch import bucketize


class MapNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None
        
class Map:
    def __init__(self):
        self.bucketSize = 10
        # Initially all the buckets will be empty, so using list comprehnsion we are making each bucket to be None
        self.buckets = [None for i in range(self.bucketSize)]
        self.count = 0
        
    def size(self):
        return self.count
    
    def getBucketIndex(self, hc):
        return (abs(hc) % (self.bucketSize))
    
    def getValue(self, key):
        # hash is an inbuilt function which reutrns an integer after hashing - hence it will return a hashcode
        hc = hash(key)
        index = self.getBucketIndex(hc)
        head = self.buckets[index]
        # checking if the index is present in the LL if yes then changing with the new value
        while head is not None:
            if head.key == key:
                return head.value
            head = head.next
        return None
    
    def remove(self, key):
        hc = hash(key)
        index = self.getBucketIndex(hc)
        head = self.buckets[index]
        prev = None
        while head is not None:
            if head.key == key:
                # In case the first element of the LL is to be removed
                if prev is None:
                    self.buckets[index] = head.next
                else:
                    prev.next = head.next
                self.count -= 1
                # Returning the value of the key which is being removed
                return head.value
            prev = head    
            head = head.next 
        return None
    
    def rehash(self):
        temp = self.buckets
        self.buckets = [None for i in range(2*self.bucketSize)]
        self.bucketSize =  2* self.bucketSize
        # As we are putting every element into the new list then we will make the count = 0 as when we are inserting it is counting
        self.count = 0
        for head in temp:
            while head is not None:
                self.insert(head.key, head.value)
                head = head.next
                
    def loadFactor(self):
        return self.count/self.bucketSize
    
    def insert(self, key, value):
        # hash is an inbuilt function which reutrns an integer after hashing - hence it will return a hashcode
        hc = hash(key)
        index = self.getBucketIndex(hc)
        # self.buckets[index] is the first position of LL
        head = self.buckets[index]
        # checking if the index is present in the LL if yes then changing with the new value
        while head is not None:
            if head.key == key:
                head.value = value
                return
            head = head.next
        head = self.buckets[index]
        # If the key is not present in the LL then adding a new node to the begenning of the LL
        newNode = MapNode(key, value)
        newNode.next = head
        self.buckets[index] = newNode
        self.count += 1     
        loadFactor = self.count/self.bucketSize
        if loadFactor >= 0.7:
            self.rehash()
            
            


In [2]:
# Example before Rehashing
m = Map()
m.insert('Parik', 2)
print(m.size())
m.insert('Rohon', 7)
print(m.size())
m.insert('Parik', 9)
print(m.size())

print(m.getValue('Rohon'))
print(m.getValue('Parik'))

print(m.remove('Rohon'))
print(m.getValue('Rohon'))
print(m.getValue('Parik'))

1
2
2
7
9
7
None
9


In [7]:
# Example after Rehashing
m = Map()
for i in range(10):
    m.insert('abc' + str(i) , i+1)
    print(m.loadFactor())
    
for i in range(10):
    print('abc' + str(i) + ":" ,m.getValue('abc' + str(i)))
# As we can see the load factor is keep on getting changed

0.1
0.2
0.3
0.4
0.5
0.6
0.35
0.4
0.45
0.5
abc0: 1
abc1: 2
abc2: 3
abc3: 4
abc4: 5
abc5: 6
abc6: 7
abc7: 8
abc8: 9
abc9: 10
